<img src="https://www.encit.unam.mx/sites/default/files/2020-10/EN%20CIENCIAS%20DE%20LA%20TIERRA1%20%281%29.png" width=200px hspace="5px" vspace="0px" style="float: left; margin-right:10px">

**Asignatura**: Herramientas Computacionales Avanzadas <br>
**Profesor**: Dr. Luis Miguel de la Cruz Salas.<br>
**Examen Práctico**: 1<br>
**Fecha**: Febrero 2024


In [ ]:
NAME = ""
COLLABORATORS = ""

---

# La derivada de una función y su aproximación

**Objetivo**.
- Revisar el concepto de derivada usando herramientas visuales que permitan comprender su sentido geométrico y comprender lo que significa el cambio instantáneo.

 <p xmlns:cc="http://creativecommons.org/ns#" xmlns:dct="http://purl.org/dc/terms/"><a property="dct:title" rel="cc:attributionURL" href="https://github.com/repomacti/macti/tree/main/notebooks/Analisis_Numerico_01">MACTI-Analisis_Numerico_01</a> by <a rel="cc:attributionURL dct:creator" property="cc:attributionName" href="https://www.macti.unam.mx">Luis M. de la Cruz</a> is licensed under <a href="http://creativecommons.org/licenses/by-sa/4.0/?ref=chooser-v1" target="_blank" rel="license noopener noreferrer" style="display:inline-block;">Attribution-ShareAlike 4.0 International<img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/cc.svg?ref=chooser-v1"><img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/by.svg?ref=chooser-v1"><img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/sa.svg?ref=chooser-v1"></a></p> 

Trabajo realizado con el apoyo del Programa UNAM-DGAPA-PAPIME PE101922

In [ ]:
# Importamos todas las bibliotecas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy as sy
import macti.visual
from macti.evaluation import *

In [ ]:
quizz = Quizz('q4', 'HeCompA')

<a name='1'></a>
## Introducción

Si revisamos con cuidado, algunas definiciones matemáticas utilizan un tipo de figura literaria conocida como <a href=https://es.wikipedia.org/wiki/Ox%C3%ADmoron>*oxímoron*</a>. En términos simples, un oxímoron consiste en usar dos conceptos de significado opuesto y con ello generar un tercer concepto. 

Por ejemplo: **La razón de cambio instantáneo**. 
- Cuando se habla de un *cambio*, se requiere de la comparación entre dos o más estados y con ello analizar las diferencias entre un estado y otro; 
- por otro lado, la palabra *instantáneo* tiene que ver con algo que dura un solo instante, es decir un tiempo puntal. 

Entonces el concepto "**cambio instantáneo**" representa un oxímoron. Pero ¿cuál es su significado?
¿Será importante este concepto en nuestra vida diaria?

En lo que sigue veremos que la razón de cambio instantáneo tiene que ver con un concepto muy importante en Cálculo: **la derivada**.

<a name='2'></a>
## La curva del olvido.

Un estudiante de lenguas participará en un concurso internacional cuyo principal reto es el conocimiento del vocabulario de un cierto idioma. Por ello, es importante que el estudiante utilice un método de estudio adecuado para recordar el significado del mayor número de palabras posible.

La <a href=https://es.wikipedia.org/wiki/Curva_del_olvido>curva del olvido</a> puede ayudar al estudiante a generar un plan de estudio adecuado. La función que define esta curva es la siguiente:

$$
R(t) = e^{-t/S}
$$

donde $R$ es cuanto recordamos, $S$ es la intensidad del recuerdo y $t$ el tiempo. Un valor de $S$ cercano a $0$ corresponde a algo que no nos interesa nada y por lo tanto se nos olvida pronto.

**Observación**: $S$ no puede ser exactamente $0$ por que en ese caso la función $R(t)$ no está definida.

La siguiente celda de código define la curva del olvido en la función `R(t, S)`:

In [ ]:
# Primero definimos la función del olvido
def R(t, S = 1.0):
    """
    Función del olvido.

    Parameters
    ----------
    t : float
    El tiempo

    S : float
    Intensidad del recuerdo. Valor por omisión S = 1.0

    Returns
    -------
    Curva del olvido evaluada en t.
    """
    return np.exp(-t/S)

La siguiente gráfica muestra cómo decrecen nuestros recuerdos con el paso del tiempo para diferentes valores de $S$.

In [ ]:
# Dominio de tiempo (hasta 7 días).
t = np.linspace(0,7,100)

# Cuatro curvas del olvido para cuatro valores de S
plt.plot(t, R(t, 0.1), lw=3, c='C4', label='{}'.format(0.1))
plt.plot(t, R(t, 1.0), lw=3, c='C3', label='{}'.format(1))
plt.plot(t, R(t, 5.0), lw=3, c='C2', label='{}'.format(5))
plt.plot(t, R(t, 10.0), lw=3, c='C1', label='{}'.format(10))

# Configuración de la gráfica. 
plt.title("Función del olvido: $R(t)=e^{-t/S}$")
plt.ylabel("$R(t)$")
plt.xlabel("$t$ [días]")
plt.legend(title = 'Intensidad $S$')
plt.grid()
plt.show()

<a name='2-1'></a>
### ¿Cuánto tiempo dura el recuerdo?

¿Será posible determinar cada cuanto tiempo el estudiante debe repasar las palabras para que no las olvide y pueda ganar el concurso? ¿De qué depende esto?

Tomemos por ejemplo el caso de $S=1.0$ (curva naranja). ¿En qué parte de la gráfica se incrementa el olvido? en otras palabras ¿en qué parte de la gráfica el descenso es más rápido?

Para conocer ese descenso, debemos calcular la pendiente $m$ y eso lo podemos hacer con la siguiente fórmula:

$$
m = \frac{R(t_2) - R(t_1)}{t_2 - t_1} \tag{1}
$$

donde $t_1$ y $t_2$ son dos tiempos distintos.

Si definimos $h = t_2 - t_1$ y $t = t_1$ podemos escribir la fórmula $(1)$ como sigue:

$$
m(t) = \frac{R(t + h) - R(t)}{h} \tag{2}
$$

En esta última fórmula vemos que la pendiente depende de $t$, es decir, en qué día nos encontramos.

Vamos a calcular $R(t)$ y $m(t)$ en $t = [0,1,2,3,4,5,6,7]$, para $h = 1$ :

In [ ]:
h = 1.0
td = np.arange(0,8,h) # Definición de las t = 0,1,2,...,7
r = np.zeros(len(td)) # Arreglo para almacenar el valor de R
m = np.zeros(len(td)) # Arreglo para almacenar las pendientes

print('td = {}'.format(td))
print('r = {}'.format(r))
print('m = {}'.format(m))

Observa que el arreglo `td` contiene los días, del $0$ al $7$, y `r` y `m` se inicializan con $0$'s.

Ahora vamos a calcular $R$ y $m$ para los diferentes valores de $t$: 

In [ ]:
# Hacemos los cálculos en cada uno de los días
for i, t in enumerate(td):
    r[i] = R(t)                  # Función del olvido
    m[i] = (R(t + h) - R(t)) / h # Pendiente, fórmula 2

# Ponemos la información en un DataFrame y la mostramos
tabla = pd.DataFrame(np.array([td, r, m]).T, 
                     columns = ['$t$', '$R(t, S=1.0)$', '$m(t)$'])
tabla

Observa que:

* el valor de la **pendiente**  $m$  es **negativa**, lo cual indica un decrecimiento,
* también la magnitud de la pendiente (su valor absoluto) disminuye conforme $t$  avanza,
* y que el recuerdo disminuye mucho al principio, de tal manera que en el tercer día solo recordamos un $4.97\%$ de lo que aprendimos en el día $0$.

Esto se ve de manera gráfica como sigue:

In [ ]:
# Dominio de tiempo (hasta 7 días).
t = np.linspace(0,7,100)

# La curva del olvido para S = 0.9
plt.plot(t, R(t), lw=3, c='C1')

# Línea punteada
plt.plot([0,1,2,3], [R(0), R(1), R(2), R(3)], 'o--', lw=1, zorder=5)

# Configuración de la gráfica. 
plt.title("Función del olvido: $R(t)=e^{-t/S}$, para $S = 1.0$")
plt.ylabel("$R(t)$")
plt.xlabel("$t$ [días]")

# Información de los primeros 3 días
plt.text(2,0.6,'$R$({2:}) = {0:5.4f},\t $m$({2:}) = {1:5.4f}'.format(R(0), m[0], 0))
plt.text(2,0.5,'$R$({2:}) = {0:5.4f},\t $m$({2:}) = {1:5.4f}'.format(R(1), m[1], 1))
plt.text(2,0.4,'$R$({2:}) = {0:5.4f},\t $m$({2:}) = {1:5.4f}'.format(R(2), m[2], 2))
plt.text(2,0.3,'$R$({2:}) = {0:5.4f},\t $m$({2:}) = {1:5.4f}'.format(R(3), m[3], 3))
plt.grid()
plt.show()

* En la figura anterior, la línea punteada une los puntos para cada uno de los días; esta línea punteada muestra gráficamente el cambio en $R(t)$ de un día a otro. 

* Entonces, lo que estamos observando es la razón de cambio de $R(t)$ en intervalos de tiempo de longitud $h = 1$. **Esto es justamente lo que expresa la fórmula $(2)$**. 

* Los valores de $R$ para los diferentes días indican como es que vamos olvidando lo que estudiamos en el día 0:
    - Para el día 1 ya solo recordamos el $36.79\%,$
    - en el segundo día el recuerdo es del $13.53\%$
    - y para el día 3 el recuerdo es mínimo, del $4.97\%$.
  
* Por lo tanto, es conveniente repasar lo aprendido en el día 0 de manera frecuente, para este caso con $S=1.0$, sería conveniente repasar todos los días.

¿Para este ejemplo, cómo podemos calcular **la razón de cambio instantáneo**? La respueta es: haciendo $h$ muy pequeña, es decir $h \to 0$. 

Para ello, esta razón debería calcularse en un solo instante de tiempo, lo cual implica que $t_1 = t_2 \Longrightarrow h = 0$, y esto nos lleva a que la fórmula de $m(t)$ no está bien definida (¡división por cero!). 

Pero, ¿qué pasa si $h$ se hace muy pequeña, pero no cero? es decir:

$$
\lim_{h \to 0}  \frac{R(t + h) - R(t)}{h} \tag{3} = ¿?
$$

¿Cúanto vale este límite? ¿Es posible calcularlo en cualquier caso y para cualquier tipo de función?

<div class="alert alert-info">

### Ejemplo 1. ¿Qué pasa cuando $h \to 0$ para diferentes valores de $S$?.

<font color="Black">
    
Ejecuta la siguiente celda de código para generar el interactivo en donde podrás modificar $S$, $h$ y $t$. Explora qué sucede para cada valor de los parámetros y posteriomente responde las preguntas del [Ejercicio 1](#quiz-1).

Para ver los valores de $R(t)$, $R^\prime(t)$ y $m(t)$ haz clic sobre el botón `Muestra valores` sobre el interactivo


</font>

</div>


In [ ]:
%run "./zinteractivo1.ipynb"

**Comentarios.**

En la gráfica de la izquierda observamos que conforme $h$ se hace más pequeño, la línea roja se aproxima cada vez mejor a la línea tangente (verde) que pasa por el punto rojo. La línea roja representa una aproximación a la razón de cambio instantánea en el punto rojo.

En la gráfica de la derecha observamos la curva $R^\prime(t)$ (verde), un punto morado que representa el valor exacto de $R^\prime(t)$ y un punto negro que es la aproximación para una $h$ dada.

Entonces, la tangente en el punto rojo, no es otra cosa que **la razón de cambio instantánea**. Veremos enseguida que ambas cosas representan un concepto conocido como **la derivada de la función** en el punto rojo.

<a name='quiz-1'></a>

<div class="alert alert-success">
<font color="black"> 

### Ejercicio 1.

El valor absoluto de la diferencia entre un valor exacto ($v_{e}$) y un valor aproximado ($v_{a}$) se conoce como el **error absoluto** y se escribe como sigue:
$$
E_a = |v_e - v_a| \tag{4}
$$
Usando esta definición y considerando que el valor exacto de la derivada es $R^\prime(t)$ y su valor aproximado es $m(t)$ realiza lo siguiente: 

1. Calcula es el error absoluto entre $R'(t)$ y $m(t)$ para $S = 1.0$, $t = 2.0$ y <br>
    1. $h = 1.0$<br>
    2. $h = 0.5$<br>
    3. $h = 0.1$<br>

Completa el código de la celda siguiente para obtener la respuesta.

**Hint:**
calcula $|v_e - v_a|$, usando la función `abs(ve - va)`.

</font>

</div>


<div class="alert alert-danger">
    
**NOTA.**
Para responder las preguntas, tienes que mover los parámetros en el interactivo a los valores correspondientes de $S$, $t$ y $h$ para obtener los valores de $R^\prime(t)$ y $m(t)$.
</div>

<div class="alert alert-success">
1.A $S = 1.0$, $t = 2.0$, $h=1.0$
</div>

In [ ]:
# Los valores ve y ve los obtienes del interactivo
# ve = ... # valor exacto R'(t) 
# va = ... # valor aproximado m(t)

# Calcula el error absoluto entre ve y va
# Ea_1 = abs(...) # Error absoluto
# YOUR CODE HERE
raise NotImplementedError()

print("R'({}, {}) = {}".format(2.0, 1.0, ve))
print("m({}, {}) = {}".format(2.0, 1.0, va))
print("Error absoluto = {}".format(Ea_1))

In [ ]:
quizz.eval_numeric('1A', Ea_1)

<div class="alert alert-success">
1.B $S = 1.0$, $t = 2.0$, $h=0.5$
</div>

In [ ]:
# Los valores ve y ve los obtienes del interactivo
# ve = ... # valor exacto R'(t) 
# va = ... # valor aproximado m(t)

# Calcula el error absoluto entre ve y va
# Ea_2 = abs(...) # Error absoluto
# YOUR CODE HERE
raise NotImplementedError()

print("R'({}, {}) = {}".format(2.0, 1.0, ve))
print("m({}, {}) = {}".format(2.0, 1.0, va))
print("Error absoluto = {}".format(Ea_2))

In [ ]:
quizz.eval_numeric('1B', Ea_2)

<div class="alert alert-success">
1.C $S = 1.0$, $t = 2.0$, $h=0.1$
</div>

In [ ]:
# Los valores ve y ve los obtienes del interactivo
# ve = ... # valor exacto R'(t) 
# va = ... # valor aproximado m(t)

# Calcula el error absoluto entre ve y va
# Ea_3 = abs(...) # Error absoluto
# YOUR CODE HERE
raise NotImplementedError()

print("R'({}, {}) = {}".format(2.0, 1.0, ve))
print("m({}, {}) = {}".format(2.0, 1.0, va))
print("Error absoluto = {}".format(Ea_3))

In [ ]:
quizz.eval_numeric('1C', Ea_3)

<div class="alert alert-success">
<font color="black"> 

### Ejercicio 2.

2. ¿Qué sucede con la diferencia entre $R^\prime(t)$ y $m(t)$, cuando $h$ se hace más pequeño ($h\to 0$), sin importar el valor de $t$ ni de $S$?
    1. Se hace más grande. <br>
    2. Se mantiene constante.
    3. Se hace más pequeña.
    4. No es posible determinarlo.

</font>
</div>

In [ ]:
# Escribe tu respuesta como sigue 
# respuesta = ...

# YOUR CODE HERE
raise NotImplementedError()


In [ ]:
quizz.eval_option('2', respuesta)

<div class="alert alert-success">
<font color="black"> 

### Ejercicio 3.

3. En las siguientes celdas escribe las siguientes funciones:
    1. `def derivada_exacta(t, S):` para calcular la derivada exacta de $R(t)$.
    2. `def derivada_aproximada(t, S, h):` para calcular la derivada aproximada de $R(t)$ usando la fórmula $(3)$.
    3. `def error_absoluto(ve, va):` para calcular el error absoluto entre un valor exacto `ve` y un valor aproximado `va`.
</font>
</div>

In [ ]:
# Completa la función de la derivada exacta de R(t)
def derivada_exacta(t, S):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Completa la función de la derivada aproximada de R(t) usando la fórmula (3)
def derivada_aproximada(t, S, h):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Completa la función de error con la fórmula (4)
def error_absoluto(ve, va):
    # YOUR CODE HERE
    raise NotImplementedError()

<div class="alert alert-success">
<font color="black"> 

### Ejercicio 4.

4. Usando las funciones definidas antes, calcula la derivada exacta $R^\prime(t)$, la derivada aproximada $m(t)$ y el error absoluto entre ellas para los siguientes casos: 
    1. $S = 0.5, t = 1, h = 1$
    2. $S = 1.0, t = 2, h = 0.1$
    3. $S = 2.0, t = 3, h = 1.0$
    4. $S = 5.0, t = 4, h = 0.1$
  
Verifica que tus resultados sean iguales a los que obtienes con el interactivo.

</font>
</div>

<div class="alert alert-success">
4.A. $S = 0.5, t = 1, h = 1$
</div>

In [ ]:
# Utiliza las funciones para calcular lo siguiente:
# ve = derivada_ ... # valor exacto R'(t) 
# va = derivada_ ... # valor aproximado m(t)
# EA = error_ ...  

S = 0.5
t = 1.0
h = 1.0
# YOUR CODE HERE
raise NotImplementedError()

print("R'({}, {}) = {}".format(t, S, ve))
print("m({}, {}, {}) = {}".format(t, S, h, va))
print("Error absoluto = {}".format(EA))

In [ ]:
quizz.eval_numeric('4A', EA)

<div class="alert alert-success">
4.B. $S = 1.0, t = 2, h = 0.1$
</div>

In [ ]:
# Utiliza las funciones para calcular lo siguiente:
# ve = derivada_ ... # valor exacto R'(t) 
# va = derivada_ ... # valor aproximado m(t)
# EB = error_ ... 

S = 1.0
t = 2.0
h = 0.1
# YOUR CODE HERE
raise NotImplementedError()

print("R'({}, {}) = {}".format(t, S, ve))
print("m({}, {}, {}) = {}".format(t, S, h, va))
print("Error absoluto = {}".format(EB))

In [ ]:
quizz.eval_numeric('4B', EB)

<div class="alert alert-success">
4.C. $S = 2.0, t = 3, h = 1.0$
</div>

In [ ]:
# Utiliza las funciones para calcular lo siguiente:
# ve = derivada_ ... # valor exacto R'(t) 
# va = derivada_ ... # valor aproximado m(t)
# EC = error_ ... 

S = 2.0
t = 3.0
h = 1.0
# YOUR CODE HERE
raise NotImplementedError()

print("R'({}, {}) = {}".format(t, S, ve))
print("m({}, {}, {}) = {}".format(t, S, h, va))
print("Error absoluto = {}".format(EC))

In [ ]:
quizz.eval_numeric('4C', EC)

<div class="alert alert-success">
4.D. $S = 5.0, t = 4, h = 0.1$
</div>

In [ ]:
# Utiliza las funciones para calcular lo siguiente:
# ve = derivada_ ... # valor exacto R'(t) 
# va = derivada_ ... # valor aproximado m(t)
# ED = error_ ... 

S = 5.0
t = 4.0
h = 0.1
# YOUR CODE HERE
raise NotImplementedError()

print("R'({}, {}) = {}".format(t, S, ve))
print("m({}, {}, {}) = {}".format(t, S, h, va))
print("Error absoluto = {}".format(ED))

In [ ]:
quizz.eval_numeric('4D', ED)

<a name='3'></a>
## Definición de derivada

La fórmula $(3)$ no es otra cosa que la definición formal de la derivada de una función. En casi todos los libros de cálculo encontrarás la siguiente notación para la derivada de la función $f(x)$:

$$ 
\frac{d f}{dx} = f^\prime(x)=\lim_{h \to 0} \frac{f(x + h) - f(x)}{h} \tag{5}
$$

La derivada existe siempre y cuando exista este límite. ¿Puedes imaginar cuando este límite no existe? 

Observe que en la definición anterior se está calculando la pendiente de la función $f(x)$ en $x$. ¿Cuándo es que esta pendiente no se puede calcular?

<div class="alert alert-info">

### Ejemplo 2. Aproximación de la derivada hacia adelante y hacia atrás.

<font color="black">

Ejecuta la siguiente celda de código. Obtendrás un interactivo en donde podrás modificar $h$ y $x$.<br>
* Explora los valores de $f^\prime$, $m$ y del Error Absoluto cuando modificas $x$ y $h$.
* Observa lo que sucede cuando activas el botón `Hacia atrás`.
* ¿El error absoluto es menor o mayor con el botón `Hacia atrás` activado? ¿De qué depende?

</font>

</div>

In [ ]:
%run "./zinteractivo2.ipynb"


Observamos en el interactivo anterior que también es posible calcular la derivada "hacia atrás" lo cual significa usar un punto a la izquierda del lugar donde se desea obtener la derivada (punto rojo). Esto se puede escribir analíticamente de la siguiente manera:

$$ 
\frac{d f}{dx} = f^\prime(x)=\lim_{h \to 0} \frac{f(x) - f(x-h)}{h} \tag{6}
$$

Entonces, las ecuaciones $(5)$ y $(6)$ indican dos maneras de calcular la derivada en un punto, pero que deben coincidir cuando $h \to 0$.

Se puede pensar en el límite por la derecha, ecuación $(5)$ y el límite por la izquierda, ecuación $(6)$. Ambos deben existir y deben ser iguales para que la derivada en un punto dado exista.

<a name='2-1'></a>
## ¿Cómo calcular la derivada analíticamente?

Consideremos la función $f(x) = x^3$ y apliquemos la definición de derivada:

$$
\frac{d f}{dx} = \lim_{h \to 0} \frac{f(x + h) - f(x)}{h} = \lim_{h \to 0} \frac{(x + h)^3 - x^3}{h}
$$

Si expandimos los términos del numerador obtenemos:

$$
\frac{d f}{dx} = \lim_{h \to 0} \, (3x^2 + 3 x h + h ^2) \tag{7}
$$

Al calcular el límite de la derecha obtenemos:

$$
\frac{d f}{dx} = 3x^2
$$

Hemos calculado la derivada analítica de $f(x) = x^3$. 


**¿Podrías calcular la derivada analíticamente usando la definición de la ecuación $(6)$?**

<div class="alert alert-success">

### Ejercicio 5. Derivada analítica hacia atrás.

<font color="black">

Usando la definición de la ecuación $(6)$ escribe la fórmula que debe ir entre paréntesis, similar a la de la ecuación $(7)$.

</font>
</div>

In [ ]:
# Escribe tu respuesta como sigue 
# respuesta = ...

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
quizz.eval_expression('5', resultado)